In [56]:
# import sklearn
# print(sklearn.__version__)

0.21.2


In [51]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
# import pickle
# import omega_index_py3 as oi
import gc
from collections import defaultdict
import os, sys, shutil
# import matplotlib.pyplot as plt
# %matplotlib inline

Each algorithm has its block to facilitate the manipulation of it if the reader wants. Thus, feel free to change it to fit your needs. We ran the CORALS algorithm (it is based on MatLab) apart and computed the scores in this file. You can compute the reconstruction error from his code just by making simple adaptations on his project. Please, check Li's paper to get access to his project code.

# Algorithms

### Setup for traditional clustering methods

In [52]:
# Used to compute the reconstruction error
syn1 = './data/synthetic/preprocessed/Toy1_b4_binary_fortraditional.dat'
syn2 = './data/synthetic/preprocessed/Toy2_b5_binary_fortraditional.dat'
syn3 = './data/synthetic/preprocessed/Toy3_b2_binary_fortraditional.dat'
files_reconstruct = [syn1, syn2, syn3]

file1 = './data/synthetic/preprocessed/Toy1_b4_binary_fortraditional.dat'#synthetic-1
file2 = './data/synthetic/preprocessed/Toy2_b5_binary_fortraditional.dat'#synthetic-2
file3 = './data/synthetic/preprocessed/Toy3_b2_binary_fortraditional.dat'#synthetic-3
datasets = [file1, file2, file3]
clusters = [6,7,3] # number of clusters that traditinal clustering found in the ground-truth, respectively.
numberOfRuns = 1

### k-means

In [53]:
from sklearn.cluster import KMeans
path_method = "OutputAnalysis\kmeans"
check_path(path_method)

for ds in range(len(datasets)):
    ds_name = "Syn-"+str(ds+1)
    print("\nDataset: "+ds_name)
    res = os.mkdir(path_method+"\\"+ds_name)

    for run in range(numberOfRuns):
        print("Run-"+str(run+1))
        df = pd.read_csv(datasets[ds],header=None)
        df.columns = [str(i) for i in range(df.shape[1])]
        ncols = df.shape[1]
        data = df.values.copy()
        del df

#         kmeans = KMeans(n_clusters = clusters[ds], random_state = 0)
        kmeans = KMeans(n_clusters = clusters[ds])
        kmeans.fit(data)
        ids_clus = list(set(kmeans.labels_))
        reconstructed_matrix = np.ones(data.shape,dtype=int)
#         print("Data cost: ",data.sum())
        print("Reconstruction error: ",np.sum(np.bitwise_xor(data,reconstructed_matrix)))
        del data, reconstructed_matrix
        gc.collect()
        
        clustering = build_clustering_output_omega(ids_clus,(kmeans.labels_,ncols),trad=True)

        # XMEASURES format ground-truth C++ version
        kmeans_clustering_xm = xmeasures_format(clustering)
        df_gt = pd.DataFrame(kmeans_clustering_xm)
#         name = datasets[ds].split("/")[-1]
#         name = name.split(".")[0]
        path = path_method+"/"+ds_name
        df_gt.to_csv(path.replace("\\","/")+"/run_"+str(run+1)+"_res_kmeans_"+ds_name+"_trad.cnl", 
                     header= False,index=False, encoding='utf8')
        del clustering, df_gt, kmeans_clustering_xm
        gc.collect()

C:\Users\yurin\Documents\my_github\ococlus
kmeans' folder was cleaned.

Dataset: Syn-1
Run-1
Reconstruction error:  6762

Dataset: Syn-2
Run-1
Reconstruction error:  368875

Dataset: Syn-3
Run-1
Reconstruction error:  3800


In [31]:
%%bash
#!/bin/bash
### omega index and f-score from Xmeasures project
### ./xmeasures -o -fp -ku -O gt.txt cls2.txt
# pwd
# ls

if [ -d "xmeasures/OutputAnalysis/kmeans" ] 
then
    echo "Directory exists. Clean it."
    rm -R xmeasures/OutputAnalysis/kmeans
else
    echo "Directory does not exists. Create it now."
fi

cp -R OutputAnalysis xmeasures
cd xmeasures/

#Methods: [Kmeans]
#ground-truth: [gt_xm_s1_trad.cnl, gt_xm_s2_trad.cnl, gt_xm_s3_trad.cnl]
# for i in 1 2 3
for i in 3
do
    for file in OutputAnalysis/kmeans/Syn-${i}/*
    do
        res=$(grep -c '' ${file})

        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./xmeasures -o -fp -ku -O ./gts/gt_xm_s${i}_trad.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
    done
    echo " "
    # wait until all child processes are done
    wait
done

Directory exists. Clean it.
 
File OutputAnalysis/kmeans/Syn-3/run_1_res_kmeans_Syn-3_trad.cnl is not empty. It has 3 lines.
 
 
= Overlaps Evaluation =
MF1p_u (PARTPROB, UNWEIGHTED):
0.44036 (Prc: 0.41129, Rec: 0.473852)
OI:
0.0623153
MF1p_u: 0.44036 (Prc: 0.41129, Rec: 0.473852); OI: 0.0623153


WARNING, the nodes in the collections differ (the quality will be penalized): 3200 nodes with hash 16332818011922301848, size: 3200, ids: 3695102400, id2s: 7185057752512) != 7000 nodes with hash 13278174547081972885, size: 7000, ids: 8209015500, id2s: 15763208793764);  synchronize: no, label: no


In [32]:
%%bash
#!/bin/bash
#### ONMI from Xmeasures project
#### ./onmi file1 file2

cd xmeasures

#Method: [kmeans]
#ground-truth: [gt_xm_s1_trad.cnl,gt_xm_s2_trad.cnl,gt_xm_s3_trad.cnl]
# for i in 1 2 3
for i in 3
do
    for file in OutputAnalysis/kmeans/Syn-${i}/*
    do
        res=$(grep -c '' ${file})
        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./onmi ./gts/gt_xm_s${i}_trad.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
#     # wait until all child processes are done
#     wait
    done
    echo " "
    # wait until all child processes are done
    wait
done

 
File OutputAnalysis/kmeans/Syn-3/run_1_res_kmeans_Syn-3_trad.cnl is not empty. It has 3 lines.
 
 
# Average estimated membership in './gts/gt_xm_s3_trad.cnl': 0.6696 (6200 / 9259)
# Average estimated membership in 'OutputAnalysis/kmeans/Syn-3/run_1_res_kmeans_Syn-3_trad.cnl': 0.6812 (7000 / 10276)
0.153246


WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 7000


### DBSCAN

In [33]:
from sklearn.cluster import DBSCAN
path_method = "OutputAnalysis\dbscan"
check_path(path_method)
for ds in range(len(datasets)):
    ds_name = "Syn-"+str(ds+1)
    print("\nDataset: "+ds_name)
    res = os.mkdir(path_method+"\\"+ds_name)

    for run in range(numberOfRuns):
        print("Run-"+str(run+1))
    
        df = pd.read_csv(datasets[ds],header=None)
        df.columns = [str(i) for i in range(df.shape[1])]
        ncols = df.shape[1]
        data = df.values.copy()
        del df

        dbscan = DBSCAN(eps = 0.5, min_samples = 5) # default parameters
        dbscan.fit(data)

        ids_clus = [ele for ele in list(set(dbscan.labels_)) if ele != -1 ]

        reconstructed_matrix = np.zeros(data.shape,dtype=int)
        for i in range(len(dbscan.labels_)):
            if dbscan.labels_[i] != -1: #mark the elements if it's not noise
                reconstructed_matrix[i,:] = 1
#         print("Data cost: ",data.sum())
        print("Reconstruction error: ",np.sum(np.bitwise_xor(data,reconstructed_matrix)))
        del data, reconstructed_matrix
        gc.collect()
        
        clustering = build_clustering_output_omega(ids_clus,(dbscan.labels_,ncols),trad=True)

        # XMEASURES format ground-truth C++ version
        dbscan_clustering_xm = xmeasures_format(clustering)
        df_gt = pd.DataFrame(dbscan_clustering_xm)
        path = path_method+"/"+ds_name
        df_gt.to_csv(path.replace("\\","/")+"/run_"+str(run+1)+"_res_dbscan_"+ds_name+"_trad.cnl", 
                     header= False,index=False, encoding='utf8')
        del clustering, df_gt, dbscan_clustering_xm
        gc.collect()

C:\Users\yurin\Documents\my_github\ococlus
The path was created: OutputAnalysis\dbscan

Dataset: Syn-1
Run-1
Reconstruction error:  6762

Dataset: Syn-2
Run-1
Reconstruction error:  368875

Dataset: Syn-3
Run-1
Reconstruction error:  3800


In [34]:
%%bash
#!/bin/bash
### omega index and f-score from Xmeasures project
### ./xmeasures -o -fp -ku -O gt.txt cls2.txt
# pwd
# ls

if [ -d "xmeasures/OutputAnalysis/dbscan" ] 
then
    echo "Directory exists. Clean it."
    rm -R xmeasures/OutputAnalysis/dbscan
else
    echo "Directory does not exists. Create it now."
fi

cp -R OutputAnalysis xmeasures
cd xmeasures/

#Methods: [Kmeans]
#ground-truth: [gt_xm_s1_trad.cnl, gt_xm_s2_trad.cnl, gt_xm_s3_trad.cnl]
# for i in 1 2 3
for i in 3
do
    for file in OutputAnalysis/dbscan/Syn-${i}/*
    do
        res=$(grep -c '' ${file})

        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./xmeasures -o -fp -ku -O ./gts/gt_xm_s${i}_trad.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
    done
    echo " "
    # wait until all child processes are done
    wait
done

Directory does not exists. Create it now.
 
File OutputAnalysis/dbscan/Syn-3/run_1_res_dbscan_Syn-3_trad.cnl is not empty. It has 3 lines.
 
 
= Overlaps Evaluation =
MF1p_u (PARTPROB, UNWEIGHTED):
0.44036 (Prc: 0.41129, Rec: 0.473852)
OI:
0.0623153
MF1p_u: 0.44036 (Prc: 0.41129, Rec: 0.473852); OI: 0.0623153


WARNING, the nodes in the collections differ (the quality will be penalized): 3200 nodes with hash 16332818011922301848, size: 3200, ids: 3695102400, id2s: 7185057752512) != 7000 nodes with hash 13278174547081972885, size: 7000, ids: 8209015500, id2s: 15763208793764);  synchronize: no, label: no


In [35]:
%%bash
#!/bin/bash
#### ONMI from Xmeasures project
#### ./onmi file1 file2

cd xmeasures

#Method: [kmeans]
#ground-truth: [gt_xm_s1_trad.cnl,gt_xm_s2_trad.cnl,gt_xm_s3_trad.cnl]
# for i in 1 2 3
for i in 3
do
    for file in OutputAnalysis/dbscan/Syn-${i}/*
    do
        res=$(grep -c '' ${file})
        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./onmi ./gts/gt_xm_s${i}_trad.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
#     # wait until all child processes are done
#     wait
    done
    echo " "
    # wait until all child processes are done
    wait
done

 
File OutputAnalysis/dbscan/Syn-3/run_1_res_dbscan_Syn-3_trad.cnl is not empty. It has 3 lines.
 
 
# Average estimated membership in './gts/gt_xm_s3_trad.cnl': 0.6696 (6200 / 9259)
# Average estimated membership in 'OutputAnalysis/dbscan/Syn-3/run_1_res_dbscan_Syn-3_trad.cnl': 0.6812 (7000 / 10276)
0.153246


WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 7000


### Hierachical clustering (Agglomerative)

In [36]:
from sklearn.cluster import AgglomerativeClustering
path_method = "OutputAnalysis\Agglomerative"
check_path(path_method)
for ds in range(len(datasets)):
    ds_name = "Syn-"+str(ds+1)
    print("\nDataset: "+ds_name)
    res = os.mkdir(path_method+"\\"+ds_name)

    for run in range(numberOfRuns):
        print("Run-"+str(run+1))
        
        df = pd.read_csv(datasets[ds],header=None)
        df.columns = [str(i) for i in range(df.shape[1])]
        ncols = df.shape[1]
        data = df.values.copy()
        del df

        HClustering = AgglomerativeClustering(n_clusters=clusters[ds], linkage='ward') #default parameters
        HClustering.fit(data)

        reconstructed_matrix = np.ones(data.shape,dtype=int)
        ids_clus = list(set(HClustering.labels_))
#         print("Data cost: ",data.sum())
        print("Reconstruction error: ",np.sum(np.bitwise_xor(data,reconstructed_matrix)))
        del reconstructed_matrix, data
        gc.collect()
        
        clustering = build_clustering_output_omega(ids_clus,(HClustering.labels_,ncols),trad=True)

        # XMEASURES format ground-truth C++ version
        HClustering_clustering_xm = xmeasures_format(clustering)
        df_gt = pd.DataFrame(HClustering_clustering_xm)
        path = path_method+"/"+ds_name
        df_gt.to_csv(path.replace("\\","/")+"/run_"+str(run+1)+"_res_agglomerative_"+ds_name+"_trad.cnl", 
                     header= False,index=False, encoding='utf8')
        del clustering, df_gt, HClustering_clustering_xm
        gc.collect()

C:\Users\yurin\Documents\my_github\ococlus
The path was created: OutputAnalysis\Agglomerative

Dataset: Syn-1
Run-1
Reconstruction error:  6762

Dataset: Syn-2
Run-1
Reconstruction error:  368875

Dataset: Syn-3
Run-1
Reconstruction error:  3800


In [37]:
%%bash
#!/bin/bash
### omega index and f-score from Xmeasures project
### ./xmeasures -o -fp -ku -O gt.txt cls2.txt
# pwd
# ls

if [ -d "xmeasures/OutputAnalysis/Agglomerative" ] 
then
    echo "Directory exists. Clean it."
    rm -R xmeasures/OutputAnalysis/Agglomerative
else
    echo "Directory does not exists. Create it now."
fi

cp -R OutputAnalysis xmeasures
cd xmeasures/

#Methods: [Kmeans]
#ground-truth: [gt_xm_s1_trad.cnl, gt_xm_s2_trad.cnl, gt_xm_s3_trad.cnl]
# for i in 1 2 3
for i in 3
do
    for file in OutputAnalysis/Agglomerative/Syn-${i}/*
    do
        res=$(grep -c '' ${file})

        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./xmeasures -o -fp -ku -O ./gts/gt_xm_s${i}_trad.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
    done
    echo " "
    # wait until all child processes are done
    wait
done

Directory does not exists. Create it now.
 
File OutputAnalysis/Agglomerative/Syn-3/run_1_res_agglomerative_Syn-3_trad.cnl is not empty. It has 3 lines.
 
 
= Overlaps Evaluation =
MF1p_u (PARTPROB, UNWEIGHTED):
0.44036 (Prc: 0.41129, Rec: 0.473852)
OI:
0.0623153
MF1p_u: 0.44036 (Prc: 0.41129, Rec: 0.473852); OI: 0.0623153


WARNING, the nodes in the collections differ (the quality will be penalized): 3200 nodes with hash 16332818011922301848, size: 3200, ids: 3695102400, id2s: 7185057752512) != 7000 nodes with hash 13278174547081972885, size: 7000, ids: 8209015500, id2s: 15763208793764);  synchronize: no, label: no


In [38]:
%%bash
#!/bin/bash
#### ONMI from Xmeasures project
#### ./onmi file1 file2

cd xmeasures

#Method: [kmeans]
#ground-truth: [gt_xm_s1_trad.cnl,gt_xm_s2_trad.cnl,gt_xm_s3_trad.cnl]
# for i in 1 2 3
for i in 3
do
    for file in OutputAnalysis/Agglomerative/Syn-${i}/*
    do
        res=$(grep -c '' ${file})
        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./onmi ./gts/gt_xm_s${i}_trad.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
#     # wait until all child processes are done
#     wait
    done
    echo " "
    # wait until all child processes are done
    wait
done

 
File OutputAnalysis/Agglomerative/Syn-3/run_1_res_agglomerative_Syn-3_trad.cnl is not empty. It has 3 lines.
 
 
# Average estimated membership in './gts/gt_xm_s3_trad.cnl': 0.6696 (6200 / 9259)
# Average estimated membership in 'OutputAnalysis/Agglomerative/Syn-3/run_1_res_agglomerative_Syn-3_trad.cnl': 0.6812 (7000 / 10276)
0.153246


WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 7000


### Setup for co-clustering methods

In [41]:
syn1 = './data/synthetic/preprocessed/Toy1_b4_binary_forcocluster.dat'
syn2 = './data/synthetic/preprocessed/Toy2_b5_binary_forcocluster.dat'
syn3 = './data/synthetic/preprocessed/Toy3_b2_binary_forcocluster.dat'
files_reconstruct = [syn1, syn2, syn3]

file1 = './data/synthetic/preprocessed/Toy1_b4_binary_forcocluster.dat'#synthetic-1
file2 = './data/synthetic/preprocessed/Toy2_b5_binary_forcocluster.dat'#synthetic-2
file3 = './data/synthetic/preprocessed/Toy3_b2_binary_forcocluster.dat'#synthetic-3
datasets = [file1, file2, file3]

### Co-clustering (Block diagonal) - Dhillon (2001)

In [42]:
from sklearn.cluster.bicluster import SpectralCoclustering
clusters = [7,10,4] # number of co-clusters in the ground-truth data, respectively.
path_method = "OutputAnalysis\dhillon"
check_path(path_method)
for ds in range(len(datasets)):
    ds_name = "Syn-"+str(ds+1)
    print("\nDataset: "+ds_name)
    res = os.mkdir(path_method+"\\"+ds_name)

    for run in range(numberOfRuns):
        print("Run-"+str(run+1))
        
        df = pd.read_csv(datasets[ds],header=None)
        df.columns = [i+1 for i in range(df.shape[1])]
        data = df.values.copy()
        del df

        DhillonCocluster = SpectralCoclustering(n_clusters = clusters[ds])
        DhillonCocluster.fit(data)

        # Reconstruct matrix
        reconstructed_matrix = np.zeros(data.shape,dtype=int)
        for nc in range(clusters[ds]):
            if len(DhillonCocluster.get_indices(nc)[0]) != 0 and len(DhillonCocluster.get_indices(nc)[1]) != 0:
                for i in DhillonCocluster.get_indices(nc)[0]:
                    for j in DhillonCocluster.get_indices(nc)[1]:
                        reconstructed_matrix[i][j] = 1
#         print("Data cost: ",data.sum())
        print("Reconstruction error: ",np.sum(np.bitwise_xor(data,reconstructed_matrix)))
        
        clustering = build_clustering_output_omega(clusters[ds],DhillonCocluster) #format to omega and f-score measure

        # XMEASURES format ground-truth C++ version
        DhillonCocluster_clustering_xm = xmeasures_format(clustering)
        df_gt = pd.DataFrame(DhillonCocluster_clustering_xm)
        path = path_method+"/"+ds_name
        df_gt.to_csv(path.replace("\\","/")+"/run_"+str(run+1)+"_res_dhillon_"+ds_name+"_co.cnl", 
                     header= False,index=False, encoding='utf8')
        del reconstructed_matrix, clustering, df_gt, DhillonCocluster_clustering_xm
        gc.collect()

C:\Users\yurin\Documents\my_github\ococlus
dhillon' folder was cleaned.

Dataset: Syn-1
Run-1
Reconstruction error:  1404

Dataset: Syn-2
Run-1
Reconstruction error:  31426

Dataset: Syn-3
Run-1
Reconstruction error:  2200


In [43]:
%%bash
#!/bin/bash
### omega index and f-score from Xmeasures project
### ./xmeasures -o -fp -ku -O gt.txt cls2.txt
# pwd
# ls

if [ -d "xmeasures/OutputAnalysis/dhillon" ] 
then
    echo "Directory exists. Clean it."
    rm -R xmeasures/OutputAnalysis/dhillon
else
    echo "Directory does not exists. Create it now."
fi

cp -R OutputAnalysis xmeasures
cd xmeasures/

#Methods: [Dhillon]
#ground-truth: [gt_xm_s1_co.cnl, gt_xm_s2_co.cnl, gt_xm_s3_co.cnl]
# for i in 1 2 3 # checking all datasets
for i in 3 # checking a specific dataset
do
    for file in OutputAnalysis/dhillon/Syn-${i}/*
    do
        res=$(grep -c '' ${file})

        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./xmeasures -o -fp -ku -O ./gts/gt_xm_s${i}_co.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
    done
    echo " "
    # wait until all child processes are done
    wait
done

Directory does not exists. Create it now.
 
File OutputAnalysis/dhillon/Syn-3/run_1_res_dhillon_Syn-3_co.cnl is not empty. It has 2 lines.
 
 
= Overlaps Evaluation =
MF1p_u (PARTPROB, UNWEIGHTED):
0.385612 (Prc: 0.325192, Rec: 0.473607)
OI:
-0.0122583
MF1p_u: 0.385612 (Prc: 0.325192, Rec: 0.473607); OI: -0.0122583


WARNING, the nodes in the collections differ (the quality will be penalized): 3200 nodes with hash 16332818011922301848, size: 3200, ids: 3695102400, id2s: 7185057752512) != 1000 nodes with hash 7130370568895277204, size: 1000, ids: 879620500, id2s: 2274756543100);  synchronize: no, label: no


In [44]:
%%bash
#!/bin/bash
#### ONMI from Xmeasures project
#### ./onmi file1 file2

cd xmeasures

#Method: [Dhillon]
#ground-truth: [gt_xm_s1_co.cnl,gt_xm_s2_co.cnl,gt_xm_s3_co.cnl]
# for i in 1 2 3 # checking all datasets
for i in 3 # checking a specific dataset
do
    for file in OutputAnalysis/dhillon/Syn-${i}/*
    do
        res=$(grep -c '' ${file})
        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./onmi ./gts/gt_xm_s${i}_co.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
#     # wait until all child processes are done
#     wait
    done
    echo " "
    # wait until all child processes are done
    wait
done

 
File OutputAnalysis/dhillon/Syn-3/run_1_res_dhillon_Syn-3_co.cnl is not empty. It has 2 lines.
 
 
# Average estimated membership in './gts/gt_xm_s3_co.cnl': 0.6696 (6200 / 9259)
# Average estimated membership in 'OutputAnalysis/dhillon/Syn-3/run_1_res_dhillon_Syn-3_co.cnl': 0.5711 (1000 / 1751)
0.156452


WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 1000


### Co-clustering (Checkerboard) - Kluger (2003)

In [46]:
from sklearn.cluster.bicluster import SpectralBiclustering
clusters = [(6,5),(7,8),(3,3)] # number of co-clusters in the ground-truth data, respectively.
path_method = "OutputAnalysis\kluger"
check_path(path_method)
for ds in range(len(datasets)):
    ds_name = "Syn-"+str(ds+1)
    print("\nDataset: "+ds_name)
    res = os.mkdir(path_method+"\\"+ds_name)

    for run in range(numberOfRuns):
        print("Run-"+str(run+1))
        
        df = pd.read_csv(datasets[ds],header=None)
        df.columns = [i+1 for i in range(df.shape[1])]
        data = df.values.copy()
        del df

        KlugerCocluster = SpectralBiclustering(n_clusters = clusters[ds])
        KlugerCocluster.fit(data)

        reconstructed_kluger = np.zeros(data.shape,dtype=int)
        for nc in range((clusters[ds][0]*clusters[ds][1])):
            if len(KlugerCocluster.get_indices(nc)[0]) != 0 and len(KlugerCocluster.get_indices(nc)[1]) != 0:
                for i in KlugerCocluster.get_indices(nc)[0]:
                    for j in KlugerCocluster.get_indices(nc)[1]:
                        reconstructed_kluger[i][j] = 1
        print("Reconstruction error: ",np.sum(np.bitwise_xor(data,reconstructed_kluger)))
        del reconstructed_kluger, data
        gc.collect()
        
        clustering = build_clustering_output_omega(clusters[ds],KlugerCocluster) #format to omega and f-score measure

        # XMEASURES format ground-truth C++ version
        KlugerCocluster_clustering_xm = xmeasures_format(clustering)
        df_gt = pd.DataFrame(KlugerCocluster_clustering_xm)
        path = path_method+"/"+ds_name
        df_gt.to_csv(path.replace("\\","/")+"/run_"+str(run+1)+"_res_kluger_"+ds_name+"_co.cnl", 
                     header= False,index=False, encoding='utf8')
        del clustering, df_gt, KlugerCocluster_clustering_xm
        gc.collect()

C:\Users\yurin\Documents\my_github\ococlus
kluger' folder was cleaned.

Dataset: Syn-1
Run-1
Reconstruction error:  3759

Dataset: Syn-2
Run-1
Reconstruction error:  343825

Dataset: Syn-3
Run-1
Reconstruction error:  1000


In [47]:
%%bash
#!/bin/bash
### omega index and f-score from Xmeasures project
### ./xmeasures -o -fp -ku -O gt.txt cls2.txt
# pwd
# ls

if [ -d "xmeasures/OutputAnalysis/kluger" ] 
then
    echo "Directory exists. Clean it."
    rm -R xmeasures/OutputAnalysis/kluger
else
    echo "Directory does not exists. Create it now."
fi

cp -R OutputAnalysis xmeasures
cd xmeasures/

#Methods: [Kluger]
#ground-truth: [gt_xm_s1_co.cnl, gt_xm_s2_co.cnl, gt_xm_s3_co.cnl]
# for i in 1 2 3 # checking all datasets
for i in 3 # checking a specific dataset
do
    for file in OutputAnalysis/kluger/Syn-${i}/*
    do
        res=$(grep -c '' ${file})

        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./xmeasures -o -fp -ku -O ./gts/gt_xm_s${i}_co.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
    done
    echo " "
    # wait until all child processes are done
    wait
done

Directory does not exists. Create it now.
 
File OutputAnalysis/kluger/Syn-3/run_1_res_kluger_Syn-3_co.cnl is not empty. It has 9 lines.
 
 
= Overlaps Evaluation =
MF1p_u (PARTPROB, UNWEIGHTED):
0.457622 (Prc: 0.580247, Rec: 0.377784)
OI:
0.0834991
MF1p_u: 0.457622 (Prc: 0.580247, Rec: 0.377784); OI: 0.0834991


WARNING, the nodes in the collections differ (the quality will be penalized): 3200 nodes with hash 16332818011922301848, size: 3200, ids: 3695102400, id2s: 7185057752512) != 4200 nodes with hash 14672793821654551294, size: 4200, ids: 4618734900, id2s: 9532372343452);  synchronize: no, label: no


In [48]:
%%bash
#!/bin/bash
#### ONMI from Xmeasures project
#### ./onmi file1 file2

cd xmeasures

#Method: [kluger]
#ground-truth: [gt_xm_s1_co.cnl,gt_xm_s2_co.cnl,gt_xm_s3_co.cnl]
# for i in 1 2 3 # checking all datasets
for i in 3 # checking a specific dataset
do
    for file in OutputAnalysis/kluger/Syn-${i}/*
    do
        res=$(grep -c '' ${file})
        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./onmi ./gts/gt_xm_s${i}_co.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
#     # wait until all child processes are done
#     wait
    done
    echo " "
    # wait until all child processes are done
    wait
done

 
File OutputAnalysis/kluger/Syn-3/run_1_res_kluger_Syn-3_co.cnl is not empty. It has 9 lines.
 
 
# Average estimated membership in './gts/gt_xm_s3_co.cnl': 0.6696 (6200 / 9259)
# Average estimated membership in 'OutputAnalysis/kluger/Syn-3/run_1_res_kluger_Syn-3_co.cnl': 0.6636 (4200 / 6329)
0.293504


WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 4200


### Li method (CORALS) - 2020

We show below the evaluation scores using the xmeasures project regarding the clustering results of Li's project code. You can compute the reconstruction error by making a simple modification on his code.

In [49]:
%%bash
#!/bin/bash
### omega index and f-score from Xmeasures project
### ./xmeasures -o -fp -ku -O gt.txt cls2.txt
# pwd
# ls

if [ -d "xmeasures/OutputAnalysis/Li" ] 
then
    echo "Directory exists. Clean it."
    rm -R xmeasures/OutputAnalysis/Li
else
    echo "Directory does not exists. Create it now."
fi

cp -R OutputAnalysis xmeasures
cd xmeasures/

#Methods: [Li]
#ground-truth: [gt_xm_s1_co.cnl,gt_xm_s2_co.cnl, gt_xm_s3_co.cnl]
# for i in 1 2 3 # checking all datasets
for i in 3 # checking a specific dataset
do
    for file in OutputAnalysis/Li/Syn-${i}/*
    do
        res=$(grep -c '' ${file})

        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
            ./xmeasures -o -fp -ku -O ./gts/gt_xm_s${i}_co.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
#     # wait until all child processes are done
#     wait
    done
    echo " "
    # wait until all child processes are done
    wait
done

Directory does not exists. Create it now.
 
File OutputAnalysis/Li/Syn-3/run_10_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
File OutputAnalysis/Li/Syn-3/run_11_result_Li_Syn-3_co.cnl
Empty file. SKIPPED!
 
File OutputAnalysis/Li/Syn-3/run_12_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
 
File OutputAnalysis/Li/Syn-3/run_13_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
 
File OutputAnalysis/Li/Syn-3/run_14_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
File OutputAnalysis/Li/Syn-3/run_15_result_Li_Syn-3_co.cnl
Empty file. SKIPPED!
 
File OutputAnalysis/Li/Syn-3/run_1_result_Li_Syn-3_co.cnl is not empty. It has 1 lines.
 
 
File OutputAnalysis/Li/Syn-3/run_2_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
 
File OutputAnalysis/Li/Syn-3/run_3_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
 
File OutputAnalysis/Li/Syn-3/run_4_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
 
File OutputAnalysis/Li/Syn-3/run_5_result_Li_Syn-3_co.cn

WARNING, the nodes in the collections differ (the quality will be penalized): 3200 nodes with hash 16332818011922301848, size: 3200, ids: 3695102400, id2s: 7185057752512) != 3551 nodes with hash 7772750386298721386, size: 3551, ids: 4246892752, id2s: 8040628001876);  synchronize: no, label: no
WARNING, the nodes in the collections differ (the quality will be penalized): 3200 nodes with hash 16332818011922301848, size: 3200, ids: 3695102400, id2s: 7185057752512) != 3257 nodes with hash 5515070431265528069, size: 3257, ids: 4019459001, id2s: 7449489215973);  synchronize: no, label: no
WARNING, the nodes in the collections differ (the quality will be penalized): 3200 nodes with hash 16332818011922301848, size: 3200, ids: 3695102400, id2s: 7185057752512) != 3489 nodes with hash 8179208330225327641, size: 3489, ids: 3940645836, id2s: 7739001506348);  synchronize: no, label: no
WARNING, the nodes in the collections differ (the quality will be penalized): 3200 nodes with hash 1633281801192230

In [50]:
%%bash
#!/bin/bash
#### ONMI from Xmeasures project
#### ./onmi file1 file2

cd xmeasures

#Methods: [Li]
#ground-truth: [gt_xm_s1_co.cnl, gt_xm_s2_co.cnl, gt_xm_s3_co.cnl]
# for i in 1 2 3 # checking all datasets
for i in 3 # checking a specific dataset
do
    for file in OutputAnalysis/Li/Syn-${i}/*
    do
    #     echo " $(grep -c '' ${file})"
        res=$(grep -c '' ${file})
    #     echo "${file} lines equal to: ${res}"

        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
    #         echo "Empty file"
            ./onmi ./gts/gt_xm_s${i}_co.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
#     # wait until all child processes are done
    wait
    done
    echo " "
    # wait until all child processes are done
#     wait
done

 
File OutputAnalysis/Li/Syn-3/run_10_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
# Average estimated membership in './gts/gt_xm_s3_co.cnl': 0.6696 (6200 / 9259)
# Average estimated membership in 'OutputAnalysis/Li/Syn-3/run_10_result_Li_Syn-3_co.cnl': 0.664 (3971 / 5980)
0.0613068
File OutputAnalysis/Li/Syn-3/run_11_result_Li_Syn-3_co.cnl
Empty file. SKIPPED!
 
File OutputAnalysis/Li/Syn-3/run_12_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
# Average estimated membership in './gts/gt_xm_s3_co.cnl': 0.6696 (6200 / 9259)
# Average estimated membership in 'OutputAnalysis/Li/Syn-3/run_12_result_Li_Syn-3_co.cnl': 0.6728 (3665 / 5447)
0.046774
 
File OutputAnalysis/Li/Syn-3/run_13_result_Li_Syn-3_co.cnl is not empty. It has 2 lines.
 
# Average estimated membership in './gts/gt_xm_s3_co.cnl': 0.6696 (6200 / 9259)
# Average estimated membership in 'OutputAnalysis/Li/Syn-3/run_13_result_Li_Syn-3_co.cnl': 0.6721 (3831 / 5700)
0.0863685
 
File OutputAnalysis/Li/Syn-3/run_14

WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 3551
WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 3257
WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 3489
WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 3968
WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 2236
WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 2687
WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 3395
WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 3601
WARNING, the number of nodes is different in the collections (the quality will be penalized): 3200 != 2365
WARNING, the number of nodes is diffe

# Support functions

### check path results

In [21]:
def check_path(path_method):
    current_dir = os.getcwd()
    print(current_dir)
    method_name = path_method.split("\\")[1]
#     print(method_name)
    res = os.path.exists(path_method)
    # clean the folder to save new data
    if res:
        #check if it is empty
        dir_empty = os.listdir(path_method)
        if len(dir_empty) != 0:
    #         shutil.rmtree("OutputAnalysis/kmeans/")
            rm = !rm -r --preserve-root './OutputAnalysis/'{method_name}'/'*
            if not rm:
                print(method_name+"' folder was cleaned.")
    #             os.chdir(path_method)
            else:
                print("sad")
                print(rm)
        else:
            print("This folder is Empty!")
            pass
    #         os.chdir(path_method)
    else: # nothing exist so create it
        # trying to insert to flase directory 
        try: 
    #         os.chdir(fd) 
            os.mkdir(path_method)
            print("The path was created: "+path_method)

        # Caching the exception     
        except: 
            print("Something wrong with specified directory. Exception- ", sys.exc_info())

### format result to evaluate

In [27]:
def build_clustering_output_omega(clusters,clus_labels,trad=False):
    # build the clustering output format to use in the omega index evaluation
    clustering = {}
    num_of_elements = 0
    
    if trad:
        for cluster_id in clusters:
            clustering["c"+str(cluster_id)] = []
            for index, value in enumerate(clus_labels[0]):#looping over the labels
                if cluster_id == value:
                    [clustering["c"+str(cluster_id)].append(("01"+str(index)+"02"+str(j))) for j in range(clus_labels[1])]# num of attributes
            num_of_elements+=len(clustering["c"+str(cluster_id)])
    else:
        if isinstance(clusters,int):
            num = clusters
        else:
            num = (clusters[0]*clusters[1])

        seq_index = 0
        for nc in range(num):
             ## check if exist some co-cluster with no element assignment for object and attribute
#             if len(clus_labels.get_indices(nc)[0]) != 0 and len(clus_labels.get_indices(nc)[1]) != 0:
#                 clustering["c"+str(seq_index)] = []
#                 for i in clus_labels.get_indices(nc)[0]:
#                     for j in clus_labels.get_indices(nc)[1]:
#                         clustering["c"+str(seq_index)].append(("01"+str(i)+"02"+str(j)))
#             elif len(clus_labels.get_indices(nc)[0]) != 0 and len(clus_labels.get_indices(nc)[1]) == 0:
#                 clustering["c"+str(seq_index)] = []
#                 for i in clus_labels.get_indices(nc)[0]:
#                     clustering["c"+str(seq_index)].append(str(i))
#             elif len(clus_labels.get_indices(nc)[1]) != 0 and len(clus_labels.get_indices(nc)[0]) == 0:
#                 clustering["c"+str(seq_index)] = []
#                 for j in clus_labels.get_indices(nc)[1]:
#                     clustering["c"+str(seq_index)].append(str(j))
#             else:
#                 pass
            
#             num_of_elements += len(clustering["c"+str(seq_index)])
#             seq_index += 1
            
            
            ## check if exist some co-cluster with no element assignment for object and attribute
            if len(clus_labels.get_indices(nc)[0]) != 0 and len(clus_labels.get_indices(nc)[1]) != 0:
                clustering["c"+str(seq_index)] = []
                for i in clus_labels.get_indices(nc)[0]:
                    for j in clus_labels.get_indices(nc)[1]:
                        clustering["c"+str(seq_index)].append(("01"+str(i)+"02"+str(j)))
                num_of_elements += len(clustering["c"+str(seq_index)])
                seq_index+=1
#         print("Number of elements in OMEGA format: ",num_of_elements)
    return clustering

In [28]:
def xmeasures_format(dict_gt):
    newData = []
#     print(len(dict_gt))
    r = 0
    for i in range(len(dict_gt)):
#         print(i,end=":")
        stringLine = str(dict_gt['c'+str(i)][0])
#         print(len(dict_gt['c'+str(i)]),end=",")
        r += len(dict_gt['c'+str(i)])
        for j in range(1,len(dict_gt['c'+str(i)])):
#             stringLine = stringLine+" "+dict_gt['c'+str(i)][j]
            stringLine += " "+str(dict_gt['c'+str(i)][j])
        newData.append(stringLine)
#     print("Number of elements in XMEASURES format: ",r)
    return newData